Atualização do inventário 08-10-2020

## Importa o inventário da rede Hidro e Telemétrica da ANA

Os dados das estações são importados do inventario do Hidro e Telemétrica da ANA, segundo o documento "DESCRIÇÃO PARA DISPONIBILIZAR DADOS HIDROMETEOROLÓGICOS DOS SISTEMAS TELEMETRIA 1 E HIDRO" da ANA, disponível em https://www.ana.gov.br/telemetria1ws/Telemetria1ws.pdf.

A importação é realizada à partir do acesso ao portal da ANA via API (Application Programming Interface), onde, através de entradas de parâmetros "vazios" é retornada uma lista com todas as estações hidrometeorológicas. Portanto, este processo pode levar algum tempo para a execução, dependendo da capacidade computacional e da conexão de internet.

Depois de gerada a lista no formato XML, o processo seguinte organiza os dados na forma de um data frame do Pandas e, então é exportado para um arquivo CSV.

OBS: caso tenha a necessidade de fazer uma personalização na importação, é recomendável ler a documentação a cima. Neste caso, os dados de entrada devem ser inseridos no dicionário `params...` e as saídas podem ser habilitadas ou excluídas na lista `columns...`.

In [1]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from tqdm import tqdm
from datetime import datetime

In [2]:
## IMPORTA INVENTÁRIO DA REDE TELEMÉTRICA ##
## Parâmetros de entrada
params = {'statusEstacoes':'', 'origem':''}
## Dados de saída
columnsStr = ['NomeEstacao', 
              'Operadora', 'Responsavel',
              'Municipio-UF', 'NomeRio',
              'Origem', 'StatusEstacao'
              ]
columnsNum = ['CodEstacao',
              'Bacia', 'SubBacia', 'CodRio',
              'Latitude', 'Longitude', 'Altitude'
              ]
## Busca no API da ANA
inventarioTelemetrica = requests.get('http://telemetriaws1.ana.gov.br/ServiceANA.asmx/ListaEstacoesTelemetricas', params)
## Organiza os dados no data frame
tree = ET.ElementTree(ET.fromstring(inventarioTelemetrica.content))
root = tree.getroot()
index = 1
dfTelemetrica = pd.DataFrame()
for station in tqdm(root.iter('Table')):
    for column in columnsStr:
        dfTelemetrica.at[index, column] = str(getattr(station.find(column), 'text', None))
    for column in columnsNum:
        dfTelemetrica.at[index, column] = getattr(station.find(column), 'text', None)
    index += 1
### Exporta os dados para o arquivo CSV
dfTelemetrica.to_csv('dfTelemetrica.csv')

4162it [00:09, 456.67it/s]


In [5]:
## IMPORTA INVENTÁRIO DA REDE HIDRO ##
## Parâmetros de entrada
params = {'codEstDE':'', 'codEstATE':'', 'tpEst':'', 'nmEst':'',
          'nmRio':'', 'codSubBacia':'', 'codBacia':'',
          'nmMunicipio':'', 'nmEstado':'',
          'sgResp':'', 'sgOper':'',
          'telemetrica':''}
## Dados de saída
columnsStr = [ 'Nome',
               'RioNome',
               'nmEstado', 'nmMunicipio',
               'ResponsavelSigla', 'OperadoraSigla',
               'ResponsavelUnidade', 'ResponsavelJurisdicao',
               'OperadoraUnidade', 'OperadoraSubUnidade',
#              'Descricao', 'NumImagens
               ]
columnsNum = [ 'Codigo', 'TipoEstacao',
               'CodigoAdicional',
#              'RioCodigo',
               'BaciaCodigo', 'SubBaciaCodigo',
#              'EstadoCodigo', 'MunicipioCodigo',
#              'ResponsavelCodigo', 'OperadoraCodigo',
               'Latitude', 'Longitude', 'Altitude', 'AreaDrenagem',
               'TipoEstacaoEscala', 'TipoEstacaoRegistradorNivel', 'TipoEstacaoDescLiquida',
               'TipoEstacaoSedimentos', 'TipoEstacaoQualAgua', 'TipoEstacaoPluviometro',
               'TipoEstacaoRegistradorChuva', 'TipoEstacaoTanqueEvapo',
               'TipoEstacaoClimatologica', 'TipoEstacaoPiezometria',
               'TipoEstacaoTelemetrica',
               'TipoRedeBasica', 'TipoRedeEnergetica', 'TipoRedeNavegacao',
               'TipoRedeCursoDagua', 'TipoRedeEstrategica', 'TipoRedeCaptacao',
               'TipoRedeSedimentos', 'TipoRedeQualAgua', 'TipoRedeClasseVazao',
               'Operando'
               ]
columnsDate = ['PeriodoEscalaInicio', 'PeriodoEscalaFim',
               'PeriodoRegistradorNivelInicio', 'PeriodoRegistradorNivelFim',
               'PeriodoDescLiquidaInicio', 'PeriodoDescLiquidaFim',
               'PeriodoSedimentosInicio', 'PeriodoSedimentosFim',
               'PeriodoQualAguaInicio', 'PeriodoQualAguaFim',
               'PeriodoPluviometroInicio', 'PeriodoPluviometroFim',
               'PeriodoRegistradorChuvaInicio', 'PeriodoRegistradorChuvaFim',
               'PeriodoTanqueEvapoInicio', 'PeriodoTanqueEvapoFim',
               'PeriodoClimatologicaInicio', 'PeriodoClimatologicaFim',
               'PeriodoPiezometriaInicio', 'PeriodoPiezometriaFim',
               'PeriodoTelemetricaInicio', 'PeriodoTelemetricaFim',
               'UltimaAtualizacao', 'DataIns', 'DataAlt'
               ]
## Busca no API da ANA
inventarioHidro = requests.get('http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroInventario', params)
## Organiza os dados no data frame
tree = ET.ElementTree(ET.fromstring(inventarioHidro.content))
root = tree.getroot()
index = 1
dfHidro = pd.DataFrame()
for station in tqdm(root.iter('Table')):
    for column in columnsStr:
        dfHidro.at[index, column] = str(getattr(station.find(column), 'text', None))
    for column in columnsNum:
        dfHidro.at[index, column] = getattr(station.find(column), 'text', None)
    for column in columnsDate:
        if station.find(column).text is not None:
            dfHidro.at[index, column] = pd.to_datetime(datetime.strptime(station.find(column).text, '%Y-%m-%d %H:%M:%S'), errors = 'coerce')
    index += 1
## Exporta os dados para o arquivo CSV
dfHidro.to_csv('dfHidro.csv')

40498it [19:17, 34.98it/s]


In [6]:
## IMPORTA A CODIFICAÇÃO DE BACIAS E SUBBACIAS ##
## Parâmetros de entrada
params = {'codBacia':'', 'codSubBacia':''}
## Dados de saída
columns = ['codBacia', 'nmBacia', 'codSubBacia', 'nmSubBacia']
## Busca no API da ANA
inventarioBacia = requests.get('http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroBaciaSubBacia', params)
## Organiza os dados no data frame
tree = ET.ElementTree(ET.fromstring(inventarioBacia.content))
root = tree.getroot()
index = 1
dfBacia = pd.DataFrame()
for station in tqdm(root.iter('Table')):
    for column in columns:
        dfBacia.at[index, column] = getattr(station.find(column), 'text', None)
    index += 1
### Exporta os dados para o arquivo CSV
dfBacia.to_csv('dfBacia.csv')

94it [00:00, 1233.70it/s]


In [7]:
## IMPORTA A CODIFICAÇÃO DE RIOS ##
## Parâmetros de entrada
params = {'codRio':''}
## Dados de saída
columns = ['BaciaCodigo', 'SubBaciaCodigo', 'Codigo', 'Nome']
## Busca no API da ANA
inventarioRio = requests.get('http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroRio', params)
## Organiza os dados no data frame
tree = ET.ElementTree(ET.fromstring(inventarioRio.content))
root = tree.getroot()
index = 1
dfRio = pd.DataFrame()
for station in tqdm(root.iter('Table')):
    for column in columns:
        dfRio.at[index, column] = getattr(station.find(column), 'text', None)
    index += 1
### Exporta os dados para o arquivo CSV
dfRio.to_csv('dfRio.csv')

13755it [00:23, 589.06it/s]


In [8]:
## IMPORTA A CODIFICAÇÃO DE ESTADOS ##
## Parâmetros de entrada
params = {'codUf':''}
## Dados de saída
columns = ['Codigo', 'CodigoIBGE', 'Sigla', 'Nome']
## Busca no API da ANA
inventarioEstado = requests.get('http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroEstado', params)
## Organiza os dados no data frame
tree = ET.ElementTree(ET.fromstring(inventarioEstado.content))
root = tree.getroot()
index = 1
dfEstado = pd.DataFrame()
for station in tqdm(root.iter('Table')):
    for column in columns:
        dfEstado.at[index, column] = getattr(station.find(column), 'text', None)
    index += 1
### Exporta os dados para o arquivo CSV
dfEstado.to_csv('dfEstado.csv')

39it [00:00, 860.88it/s]


In [9]:
## IMPORTA A CODIFICAÇÃO DE MUNICIPIOS ##
## Parâmetros de entrada
params = {'codMunicipio':''}
## Dados de saída
columns = ['EstadoCodigo', 'Codigo', 'CodigoIBGE', 'Nome']
## Busca no API da ANA
inventarioMunicipio = requests.get('http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroMunicipio', params)
## Organiza os dados no data frame
tree = ET.ElementTree(ET.fromstring(inventarioMunicipio.content))
root = tree.getroot()
index = 1
dfMunicipio = pd.DataFrame()
for station in tqdm(root.iter('Table')):
    for column in columns:
        dfMunicipio.at[index, column] = getattr(station.find(column), 'text', None)
    index += 1
### Exporta os dados para o arquivo CSV
dfMunicipio.to_csv('dfMunicipio.csv')

5710it [00:05, 1096.99it/s]


In [10]:
## IMPORTA A CODIFICAÇÃO DE ENTIDADES ##
## Parâmetros de entrada
params = {'codEntidade':''}
## Dados de saída
columns = ['Codigo', 'Sigla', 'Nome']
## Busca no API da ANA
inventarioEntidades = requests.get('http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroEntidades', params)
## Organiza os dados no data frame
tree = ET.ElementTree(ET.fromstring(inventarioEntidades.content))
root = tree.getroot()
index = 1
dfEntidade = pd.DataFrame()
for station in tqdm(root.iter('Table')):
    for column in columns:
        dfEntidade.at[index, column] = getattr(station.find(column), 'text', None)
    index += 1
### Exporta os dados para o arquivo CSV
dfEntidade.to_csv('dfEntidade.csv')

1010it [00:00, 1586.67it/s]
